
#Homework 5 - Question 1

The problem is to plan production in order to meet demand and maximize profit. There is a holding cost for units left in inventory for one period to the next. Inventory in a period cannot exceed the storage capacity. The inventory at the end of the planning horizon should be the same as the initial inventory.

**Step 1: Set up and import Gurobi**

In [ ]:
# Install Gurobi
!pip install gurobipy

# Import the Gurobi Package
import gurobipy as gp
from gurobipy import GRB



**Step 2: Read Excel Data**

In [ ]:
from openpyxl import load_workbook

# Read worksheets from spreadsheet
wb = load_workbook("production_planning_data.xlsx")
ws = wb["Data"]

# Create list with periods
periods = [ws.cell(r,1).value for r in range(2,ws.max_row + 1)]

# Create a dictionary for the capacities
capacity = {ws.cell(r,1).value: ws.cell(r,2).value
     for r in range(2,ws.max_row + 1)
}

# Create a dictionary for the demands
demand = {ws.cell(r,1).value: ws.cell(r,3).value
     for r in range(2,ws.max_row + 1)
}

# Create a dictionary for the costs
cost = {ws.cell(r,1).value: ws.cell(r,4).value
        for r in range(2,ws.max_row + 1)
}

# Create a dictionary for the prices
price = {ws.cell(r,1).value: ws.cell(r,5).value
     for r in range(2,ws.max_row + 1)
}

In [ ]:
print(periods)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25]


In [ ]:
print(capacity)

{1: 1920, 2: 1532, 3: 1667, 4: 1412, 5: 1240, 6: 1797, 7: 1999, 8: 1435, 9: 1329, 10: 1466, 11: 1076, 12: 1877, 13: 1772, 14: 2014, 15: 1539, 16: 2377, 17: 2979, 18: 2755, 19: 1654, 20: 1942, 21: 2148, 22: 1976, 23: 2189, 24: 1820, 25: 2727}


In [ ]:
print(demand)

{1: 1146, 2: 2045, 3: 1502, 4: 2464, 5: 1503, 6: 2264, 7: 2258, 8: 2470, 9: 1785, 10: 1958, 11: 1328, 12: 1205, 13: 1626, 14: 1123, 15: 2452, 16: 1573, 17: 1353, 18: 1871, 19: 1563, 20: 1857, 21: 1531, 22: 1165, 23: 1325, 24: 2325, 25: 1321}


In [ ]:
print(cost)

{1: 39, 2: 38, 3: 59, 4: 38, 5: 53, 6: 48, 7: 54, 8: 59, 9: 55, 10: 48, 11: 41, 12: 27, 13: 29, 14: 59, 15: 45, 16: 36, 17: 45, 18: 49, 19: 26, 20: 21, 21: 53, 22: 42, 23: 21, 24: 40, 25: 25}


In [ ]:
print(price)

{1: 89, 2: 74, 3: 64, 4: 95, 5: 61, 6: 82, 7: 63, 8: 69, 9: 71, 10: 84, 11: 83, 12: 98, 13: 63, 14: 93, 15: 79, 16: 99, 17: 86, 18: 63, 19: 65, 20: 77, 21: 80, 22: 81, 23: 88, 24: 81, 25: 63}


**Step 3: Set up the Linear Programming Model**

In [ ]:
# Create a Gurobi model
m = gp.Model("Plan_Production")

# Define the decision variables
x = m.addVars(periods,name='Production')
y = m.addVars(periods,name='Inventory')
z = m.addVars(periods,name='Sales')

# Define the objective function
m.setObjective(sum((z[j]*price[j])-(x[j]*cost[j])-(y[j]*5) for j in periods), sense=GRB.MAXIMIZE)

# Define the constraints

m.addConstrs(x[i] <= capacity[i] for i in periods)
m.addConstrs(y[i] <= 500 for i in periods)
m.addConstrs(z[i] <= demand[i] for i in periods)

m.addConstrs(y[i] == 500 + x[i] - z[i] if i==1 else y[i] == y[i-1] + x[i] - z[i] for i in periods)
m.addConstrs(y[25] == 500 for i in periods)

#m.addConstrs(x[i] >= 0 for i in periods)
#m.addConstrs(y[i] >= 0 for i in periods)


{1: <gurobi.Constr *Awaiting Model Update*>,
 2: <gurobi.Constr *Awaiting Model Update*>,
 3: <gurobi.Constr *Awaiting Model Update*>,
 4: <gurobi.Constr *Awaiting Model Update*>,
 5: <gurobi.Constr *Awaiting Model Update*>,
 6: <gurobi.Constr *Awaiting Model Update*>,
 7: <gurobi.Constr *Awaiting Model Update*>,
 8: <gurobi.Constr *Awaiting Model Update*>,
 9: <gurobi.Constr *Awaiting Model Update*>,
 10: <gurobi.Constr *Awaiting Model Update*>,
 11: <gurobi.Constr *Awaiting Model Update*>,
 12: <gurobi.Constr *Awaiting Model Update*>,
 13: <gurobi.Constr *Awaiting Model Update*>,
 14: <gurobi.Constr *Awaiting Model Update*>,
 15: <gurobi.Constr *Awaiting Model Update*>,
 16: <gurobi.Constr *Awaiting Model Update*>,
 17: <gurobi.Constr *Awaiting Model Update*>,
 18: <gurobi.Constr *Awaiting Model Update*>,
 19: <gurobi.Constr *Awaiting Model Update*>,
 20: <gurobi.Constr *Awaiting Model Update*>,
 21: <gurobi.Constr *Awaiting Model Update*>,
 22: <gurobi.Constr *Awaiting Model Update*

**Step 4: Solve the Model and Display Output**

In [ ]:
# Solve the model
m.optimize()

# Display the results
print(f"Maximum Profit: ${m.ObjVal:.2f}")

# Display the optimal production and inventory levels in each period
for i in periods:
  print(f"In Period {i}, demand met for {z[i].x} units by producing {x[i].x} units and storing {y[i].x} units in inventory at the end the period." )


Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (linux64)

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 125 rows, 75 columns and 199 nonzeros
Model fingerprint: 0xad7b905e
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [5e+00, 1e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e+02, 3e+03]
Presolve removed 115 rows and 49 columns
Presolve time: 0.01s
Presolved: 10 rows, 26 columns, 31 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.9456962e+06   1.460303e+03   0.000000e+00      0s
      11    1.3979930e+06   0.000000e+00   0.000000e+00      0s

Solved in 11 iterations and 0.02 seconds (0.00 work units)
Optimal objective  1.397993000e+06
Maximum Profit: $1397993.00
In Period 1, demand met for 1146.0 units by producing 1146.0 units and storing 500.0 units in inventory at the end t